In [ ]:
# !pip install mlflow

In [ ]:
# !pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.9 MB/s eta 0:00:00


In [ ]:
# !pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
# !pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [2]:
import mlflow
import mlflow.sklearn
import mlflow.data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score
# import boto3
import os
import yaml
import json
from pathlib import Path
from datetime import datetime
import hashlib
import joblib
from mlflow.tracking import MlflowClient
from typing import Optional, Dict, Any, Tuple
import optuna
# from optuna.integration import MLflowCallback
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier


/home/krish/miniconda3/envs/mlflow/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.metrics import make_scorer

In [4]:
# Setup MLflow tracking
mlflow.set_tracking_uri("./mlruns")
experiment_name = "ml-pipeline-versioning-experiment-1"
mlflow.set_experiment(experiment_name)

2025/06/09 11:09:42 INFO mlflow.tracking.fluent: Experiment with name 'ml-pipeline-versioning-experiment-1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/krish/Music/mlflow/mlruns/134065526802954240', creation_time=1749447582095, experiment_id='134065526802954240', last_update_time=1749447582095, lifecycle_stage='active', name='ml-pipeline-versioning-experiment-1', tags={}>

In [5]:
import pandas as pd
from sklearn.datasets import load_iris

# Load the iris dataset
iris = load_iris()

# Convert to pandas DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target variable
iris_df['target'] = iris.target

# Add the target names
iris_df['target_names'] = iris_df['target'].apply(lambda x: iris.target_names[x])

print(iris_df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target target_names  
0       0       setosa  
1       0       setosa  
2       0       setosa  
3       0       setosa  
4       0       setosa  


In [6]:
X= iris_df.drop(['target', 'target_names'], axis=1)
y= iris_df['target']

In [7]:
class DataVersioningManager:
    """Manages data versioning and storage"""

    def __init__(self, base_path="data_versions"):
        self.base_path = base_path
        self.client = MlflowClient()
        os.makedirs(base_path, exist_ok=True)

    def create_data_version(self, data: pd.DataFrame, version_name: str = None,
                           source: str = "unknown", tags: Dict = None) -> str:
        """Create a new data version"""

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        data_hash = self._calculate_data_hash(data)

        if version_name is None:
            version_name = f"data_v_{timestamp}_{data_hash[:8]}"

        # Create data version metadata
        data_metadata = {
            "version_name": version_name,
            "timestamp": timestamp,
            "data_hash": data_hash,
            "shape": data.shape,
            "columns": list(data.columns),
            "source": source,
            "tags": tags or {}
        }

        # Save data and metadata
        data_path = os.path.join(self.base_path, f"{version_name}.csv")
        metadata_path = os.path.join(self.base_path, f"{version_name}_metadata.json")

        data.to_csv(data_path, index=False)
        with open(metadata_path, 'w') as f:
            json.dump(data_metadata, f, indent=2)

        print(f"✅ Data version created: {version_name}")
        print(f"   Shape: {data.shape}")
        print(f"   Hash: {data_hash[:16]}...")
        print(f"   Source: {source}")

        return version_name

    def load_data_version(self, version_name: str) -> Tuple[pd.DataFrame, Dict]:
        """Load a specific data version"""

        data_path = os.path.join(self.base_path, f"{version_name}.csv")
        metadata_path = os.path.join(self.base_path, f"{version_name}_metadata.json")

        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data version {version_name} not found")

        data = pd.read_csv(data_path)

        if os.path.exists(metadata_path):
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
        else:
            metadata = {}

        return data, metadata

    def list_data_versions(self) -> pd.DataFrame:
        """List all available data versions"""

        versions = []
        for file in os.listdir(self.base_path):
            if file.endswith("_metadata.json"):
                with open(os.path.join(self.base_path, file), 'r') as f:
                    metadata = json.load(f)
                    versions.append(metadata)

        if versions:
            return pd.DataFrame(versions)
        else:
            return pd.DataFrame()

    def merge_data_versions(self, version_names: list, new_version_name: str = None) -> str:
        """Merge multiple data versions"""

        combined_data = []
        sources = []

        for version_name in version_names:
            data, metadata = self.load_data_version(version_name)
            combined_data.append(data)
            sources.append(metadata.get('source', version_name))

        merged_data = pd.concat(combined_data, ignore_index=True)
        merged_source = f"merged_from_{'+'.join(sources)}"

        new_version = self.create_data_version(
            merged_data,
            new_version_name,
            merged_source,
            {"merged_from": version_names}
        )

        return new_version

    def _calculate_data_hash(self, data: pd.DataFrame) -> str:
        """Calculate hash of data for versioning"""
        return hashlib.md5(data.to_string().encode()).hexdigest()


In [8]:
class ModelVersioningManager:
    """Manages model versioning and registry"""

    def __init__(self, model_name="pipeline_classifier"):
        self.model_name = model_name
        self.client = MlflowClient()

    def register_model_version(self, run_id: str, model_path: str = "model",
                              description: str = None, tags: Dict = None) -> int:
        """Register a new model version"""

        model_uri = f"runs:/{run_id}/{model_path}"

        # Register model version
        model_version = mlflow.register_model(
            model_uri=model_uri,
            name=self.model_name,
            tags=tags
        )

        # Add description if provided
        if description:
            self.client.update_model_version(
                name=self.model_name,
                version=model_version.version,
                description=description
            )

        print(f"✅ Model version {model_version.version} registered")
        return int(model_version.version)

    def get_latest_model_version(self) -> int:
        """Get the latest model version number"""
        try:
            latest_versions = self.client.get_latest_versions(
                name=self.model_name,
                stages=["None", "Staging", "Production"]
            )
            if latest_versions:
                return max([int(v.version) for v in latest_versions])
            return 0
        except:
            return 0

    def load_model_version(self, version: int = None):
        """Load a specific model version"""

        if version is None:
            version = self.get_latest_model_version()

        model_uri = f"models:/{self.model_name}/{version}"
        model = mlflow.sklearn.load_model(model_uri)

        print(f"✅ Loaded model version {version}")
        return model

    def promote_model_version(self, version: int, stage: str):
        """Promote model version to a stage (Staging/Production)"""

        self.client.transition_model_version_stage(
            name=self.model_name,
            version=version,
            stage=stage
        )

        print(f"✅ Model version {version} promoted to {stage}")

    def list_model_versions(self) -> pd.DataFrame:
        """List all model versions"""

        try:
            versions = self.client.search_model_versions(f"name='{self.model_name}'")
            version_data = []

            for version in versions:
                version_data.append({
                    'version': version.version,
                    'stage': version.current_stage,
                    'creation_timestamp': version.creation_timestamp,
                    'run_id': version.run_id,
                    'description': version.description
                })

            return pd.DataFrame(version_data)
        except:
            return pd.DataFrame()


In [18]:
class MLflowVersioningPipeline:
    """Enhanced MLflow Pipeline with Data and Model Versioning"""

    def __init__(self, experiment_name="ml-pipeline-versioning-experiment",
                 model_name="pipeline_classifier"):
        self.experiment_name = experiment_name
        self.model_name = model_name
        mlflow.set_experiment(experiment_name)

        self.num_classes = None
        self.data_manager = DataVersioningManager()
        self.model_manager = ModelVersioningManager(model_name)
        self.artifact_path = "pipeline_artifacts"

    def create_initial_data_version(self, data: pd.DataFrame, version_name: str = None) -> str:
        """Create initial data version"""

        if version_name is None:
            version_name = "initial_training_data_v1"

        return self.data_manager.create_data_version(
            data, version_name, "initial_training", {"type": "training"}
        )

    def _get_metric_function(self, metric_name: str, for_cv: bool = False):
        """Get metric function by name"""

        metric_name = metric_name.lower()

        average = 'binary' if self.num_classes == 2 else 'macro'

        if metric_name == 'accuracy':
            return make_scorer(accuracy_score) if for_cv else accuracy_score
        elif metric_name == 'precision':
            return make_scorer(precision_score, average=average) if for_cv else \
                  lambda y_true, y_pred: precision_score(y_true, y_pred, average=average)
        elif metric_name == 'recall':
            return make_scorer(recall_score, average=average) if for_cv else \
                  lambda y_true, y_pred: recall_score(y_true, y_pred, average=average)
        elif metric_name == 'f1':
            return make_scorer(f1_score, average=average) if for_cv else \
                  lambda y_true, y_pred: f1_score(y_true, y_pred, average=average)
        elif metric_name == 'roc_auc':
            return make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr') if for_cv else \
                  lambda y_true, y_prob: roc_auc_score(y_true, y_prob, multi_class='ovr')
        else:
            raise ValueError(f"Unsupported metric: {metric_name}")



    def _create_model_instance(self, model_name: str, params: Dict = None):
        """Create model instance with parameters"""
        model_classes = {
            'randomforest': RandomForestClassifier,
            'xgboost': xgb.XGBClassifier,
            'lightgbm': lgb.LGBMClassifier,
            'catboost': CatBoostClassifier
        }

        model_class = model_classes.get(model_name.lower())
        if model_class is None:
            raise ValueError(f"Model {model_name} not supported")

        if params is None:
            params = {}

        # Handle CatBoost verbose parameter
        if model_name.lower() == 'catboost':
            params.setdefault('verbose', False)

        return model_class(**params)

    def _optimize_hyperparameters(self, X_train, y_train, model_name: str,
                                param_space: Dict, metric: str, n_trials: int = 100):
        """Optimize hyperparameters using Optuna"""

        def objective(trial):
            # Sample parameters from the space
            params = {}
            for param_name, param_config in param_space.items():
                if param_config['type'] == 'int':
                    params[param_name] = trial.suggest_int(
                        param_name, param_config['low'], param_config['high']
                    )
                elif param_config['type'] == 'float':
                    params[param_name] = trial.suggest_float(
                        param_name, param_config['low'], param_config['high']
                    )
                elif param_config['type'] == 'categorical':
                    params[param_name] = trial.suggest_categorical(
                        param_name, param_config['choices']
                    )

            # Create and train model
            model = self._create_model_instance(model_name, params)

            scorer = self._get_metric_function(metric, for_cv=True)

            # Cross-validation score
            scores = cross_val_score(
                model, X_train, y_train,
                cv=5, scoring=scorer
            )

            return scores.mean()

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)

        return study.best_params, study.best_value

    def run_training_pipeline(self, X_train, y_train, X_test, y_test,
                             models_dict: Dict, hyperparams_dict: Dict = None,
                             metric: str = 'accuracy', n_trials: int = 100,
                             data_version: str = None, description: str = None) -> Dict:
        """Run training pipeline with model selection and hyperparameter tuning"""

        with mlflow.start_run(run_name=f"training_pipeline_model_selection"):
            print(f"🚀 Starting Training Pipeline with Model Selection")

            # Log basic info
            mlflow.log_param("train_shape", X_train.shape)
            mlflow.log_param("test_shape", X_test.shape)
            mlflow.log_param("metric", metric)
            mlflow.log_param("n_trials", n_trials)

            self.num_classes = len(np.unique(y_train))

            if data_version:
                mlflow.log_param("data_version", data_version)

            # Preprocessing
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            best_model = None
            best_score = -np.inf
            best_model_name = None
            best_params = None
            model_results = {}

            # Test each model
            for model_name in models_dict.keys():
                print(f"\n🔍 Testing model: {model_name}")

                # Get hyperparameter space for this model
                param_space = hyperparams_dict.get(model_name, {}) if hyperparams_dict else {}

                if param_space:
                    # Optimize hyperparameters
                    print(f"   Optimizing hyperparameters with {n_trials} trials...")
                    best_model_params, best_cv_score = self._optimize_hyperparameters(
                        X_train_scaled, y_train, model_name, param_space, metric, n_trials
                    )
                    print(f"   Best CV score: {best_cv_score:.4f}")
                else:
                    # Use default parameters
                    best_model_params = {}
                    best_cv_score = 0

                # Train final model with best parameters
                model = self._create_model_instance(model_name, best_model_params)
                model.fit(X_train_scaled, y_train)

                # Evaluate
                train_pred = model.predict(X_train_scaled)
                test_pred = model.predict(X_test_scaled)

                metric_func = self._get_metric_function(metric, for_cv=False)

                if metric.lower() == 'roc_auc':
                    train_score = metric_func(y_train, model.predict_proba(X_train_scaled)[:, 1])
                    test_score = metric_func(y_test, model.predict_proba(X_test_scaled)[:, 1])
                else:
                    train_score = metric_func(y_train, train_pred)
                    test_score = metric_func(y_test, test_pred)

                # Log model results
                mlflow.log_param(f"{model_name}_best_params", best_model_params)
                mlflow.log_metric(f"{model_name}_train_{metric}", train_score)
                mlflow.log_metric(f"{model_name}_test_{metric}", test_score)

                model_results[model_name] = {
                    'model': model,
                    'params': best_model_params,
                    'train_score': train_score,
                    'test_score': test_score,
                    'cv_score': best_cv_score
                }

                print(f"   Train {metric}: {train_score:.4f}")
                print(f"   Test {metric}: {test_score:.4f}")

                # Check if this is the best model
                if test_score > best_score:
                    best_score = test_score
                    best_model = model
                    best_model_name = model_name
                    best_params = best_model_params

            print(f"\n🏆 Best model: {best_model_name} with {metric} = {best_score:.4f}")

            # Log best model info
            mlflow.log_param("best_model", best_model_name)
            mlflow.log_param("best_params", best_params)
            mlflow.log_metric(f"best_{metric}", best_score)

            # Retrain best model on full dataset
            print(f"\n🔄 Retraining best model on full dataset...")
            X_full = np.vstack([X_train_scaled, X_test_scaled])
            y_full = np.concatenate([y_train, y_test])

            final_model = self._create_model_instance(best_model_name, best_params)
            final_model.fit(X_full, y_full)

            # Log final model
            mlflow.sklearn.log_model(final_model, "model")

            # Log scaler
            joblib.dump(scaler, "scaler.pkl")
            mlflow.log_artifact("scaler.pkl", self.artifact_path)
            os.remove("scaler.pkl")

            # Log model results summary
            results_df = pd.DataFrame(model_results).T
            results_df.to_csv("model_comparison.csv")
            mlflow.log_artifact("model_comparison.csv", self.artifact_path)
            os.remove("model_comparison.csv")

            # Register model version
            run_id = mlflow.active_run().info.run_id
            model_version = self.model_manager.register_model_version(
                run_id,
                description=description or f"Best model: {best_model_name}"
            )

            mlflow.log_param("model_version", model_version)
            mlflow.set_tag("model_version", model_version)
            mlflow.set_tag("best_model", best_model_name)

            print(f"✅ Training completed!")
            print(f"   Best Model: {best_model_name}")
            print(f"   Model Version: {model_version}")
            print(f"   Best {metric}: {best_score:.4f}")

            return {
                'run_id': run_id,
                'model_version': model_version,
                'best_model_name': best_model_name,
                'best_params': best_params,
                'best_score': best_score,
                'metric': metric,
                'final_model': final_model,
                'scaler': scaler,
                'model_results': model_results
            }

    def make_predictions_with_logging(self, X_new: pd.DataFrame,
                                     model_version: int = None,
                                     log_predictions: bool = True) -> Tuple[np.ndarray, str]:
        """Make predictions and optionally log them for future retraining"""

        # Load model
        if model_version is None:
            model_version = self.model_manager.get_latest_model_version()

        model = self.model_manager.load_model_version(model_version)

        # Make predictions
        predictions = model.predict(X_new)
        probabilities = model.predict_proba(X_new)

        # Log predictions if requested
        prediction_data_version = None
        if log_predictions:
            # Create prediction data with features
            pred_data = X_new.copy()
            pred_data['prediction'] = predictions
            pred_data['prediction_confidence'] = np.max(probabilities, axis=1)
            pred_data['timestamp'] = datetime.now().isoformat()

            # Create data version for predictions
            prediction_data_version = self.data_manager.create_data_version(
                pred_data,
                source=f"predictions_model_v{model_version}",
                tags={"type": "predictions", "model_version": model_version}
            )

        print(f"✅ Made {len(predictions)} predictions using model version {model_version}")
        if prediction_data_version:
            print(f"   Predictions logged as data version: {prediction_data_version}")

        return predictions, prediction_data_version

    def retrain_with_new_data(self, new_data: pd.DataFrame,
                             include_prediction_data: bool = True,
                             models_dict: Dict = None,
                             hyperparams_dict: Dict = None,
                             metric: str = 'accuracy') -> Dict:
        """Retrain model with new data combined with existing data"""

        with mlflow.start_run(run_name="retraining_pipeline"):
            print("🔄 Starting Retraining Pipeline...")

            # Get all available data versions
            available_versions = self.data_manager.list_data_versions()

            if available_versions.empty:
                print("⚠️  No previous data versions found. Creating initial version.")
                initial_version = self.create_initial_data_version(new_data)

                # Prepare data for training
                if 'target' in new_data.columns:
                    X = new_data.drop('target', axis=1)
                    y = new_data['target']
                else:
                    X = new_data.iloc[:, :-1]
                    y = new_data.iloc[:, -1]

                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, random_state=42, stratify=y
                )

                return self.run_training_pipeline(
                    X_train, y_train, X_test, y_test,
                    models_dict, hyperparams_dict, metric
                )

            # Create version for new data
            new_data_version = self.data_manager.create_data_version(
                new_data,
                source="new_training_data",
                tags={"type": "retraining"}
            )

            # Get versions to merge
            versions_to_merge = []

            # Include original training data
            training_versions = available_versions[
                available_versions['tags'].apply(
                    lambda x: isinstance(x, dict) and x.get('type') == 'training'
                )
            ]
            if not training_versions.empty:
                versions_to_merge.extend(training_versions['version_name'].tolist())

            # Include prediction data if requested
            if include_prediction_data:
                prediction_versions = available_versions[
                    available_versions['tags'].apply(
                        lambda x: isinstance(x, dict) and x.get('type') == 'predictions'
                    )
                ]
                if not prediction_versions.empty:
                    versions_to_merge.extend(prediction_versions['version_name'].tolist())

            # Add new data version
            versions_to_merge.append(new_data_version)

            # Merge all data versions
            merged_version = self.data_manager.merge_data_versions(
                versions_to_merge,
                f"merged_retraining_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            )

            # Load merged data
            merged_data, _ = self.data_manager.load_data_version(merged_version)

            # Prepare data for training
            if 'target' in merged_data.columns:
                X = merged_data.drop('target', axis=1)
                y = merged_data['target']
            else:
                X = merged_data.iloc[:, :-1]
                y = merged_data.iloc[:, -1]

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=y
            )

            # Log retraining info
            mlflow.log_param("retraining", True)
            mlflow.log_param("merged_versions", len(versions_to_merge))
            mlflow.log_param("versions_merged", versions_to_merge)
            mlflow.log_param("include_predictions", include_prediction_data)

            # Run training on merged data
            training_result = self.run_training_pipeline(
                X_train, y_train, X_test, y_test,
                models_dict, hyperparams_dict, metric,
                data_version=merged_version,
                description=f"Retrained on {len(versions_to_merge)} data versions"
            )

            # Compare with previous model
            previous_version = self.model_manager.get_latest_model_version() - 1
            if previous_version > 0:
                print(f"📊 Comparing with previous model version {previous_version}")
                mlflow.log_param("previous_model_version", previous_version)

            print(f"✅ Retraining completed!")
            print(f"   New Model Version: {training_result['model_version']}")
            print(f"   Data Versions Merged: {len(versions_to_merge)}")

            return training_result

    def setup_feedback_loop(self, X_new: pd.DataFrame, y_true: np.ndarray = None,
                           auto_retrain_threshold: float = 0.1) -> Dict:
        """Setup feedback loop for continuous learning"""

        print("🔄 Setting up Feedback Loop...")

        # Make predictions
        predictions, pred_data_version = self.make_predictions_with_logging(X_new)

        results = {
            'predictions': predictions,
            'prediction_data_version': pred_data_version
        }

        # If ground truth is available, evaluate and potentially retrain
        if y_true is not None:
            current_accuracy = accuracy_score(y_true, predictions)
            print(f"📊 Current Model Accuracy: {current_accuracy:.4f}")

            # Get previous model performance
            latest_version = self.model_manager.get_latest_model_version()
            model_versions_df = self.model_manager.list_model_versions()

            if not model_versions_df.empty:
                # Get previous accuracy from run metrics
                previous_run_id = model_versions_df[
                    model_versions_df['version'] == str(latest_version)
                ]['run_id'].iloc[0]

                try:
                    previous_run = mlflow.get_run(previous_run_id)
                    previous_accuracy = previous_run.data.metrics.get('test_accuracy', 0)

                    accuracy_drop = previous_accuracy - current_accuracy
                    print(f"📊 Previous Model Accuracy: {previous_accuracy:.4f}")
                    print(f"📊 Accuracy Drop: {accuracy_drop:.4f}")

                    # Auto-retrain if performance drops significantly
                    if accuracy_drop > auto_retrain_threshold:
                        print(f"🚨 Performance dropped by {accuracy_drop:.4f}, triggering retraining...")

                        # Create labeled data for retraining
                        labeled_data = X_new.copy()
                        labeled_data['target'] = y_true

                        retrain_result = self.retrain_with_new_data(labeled_data)
                        results.update(retrain_result)
                        results['retraining_triggered'] = True
                    else:
                        print("✅ Model performance is stable, no retraining needed")
                        results['retraining_triggered'] = False

                except Exception as e:
                    print(f"⚠️  Could not compare with previous model: {e}")

            results['current_accuracy'] = current_accuracy

        return results

    def get_pipeline_status(self) -> Dict:
        """Get comprehensive pipeline status"""

        status = {}

        # Data versions
        data_versions = self.data_manager.list_data_versions()
        status['data_versions'] = {
            'total': len(data_versions),
            'latest': data_versions.iloc[-1]['version_name'] if not data_versions.empty else None
        }

        # Model versions
        model_versions = self.model_manager.list_model_versions()
        status['model_versions'] = {
            'total': len(model_versions),
            'latest': self.model_manager.get_latest_model_version(),
            'production': None
        }

        # Check for production model
        if not model_versions.empty:
            prod_models = model_versions[model_versions['stage'] == 'Production']
            if not prod_models.empty:
                status['model_versions']['production'] = prod_models.iloc[0]['version']

        return status


In [19]:
X_train, X_test, y_train , y_test = train_test_split(X,y)

In [22]:
model_name='test-model'

In [30]:
initial_data_version_name = pipeline.create_initial_data_version(
    data=pd.concat([X, y], axis=1), # Combine X and y for data versioning
    version_name="initial_production_training_data_v1"
)
print(f"Initial data version created: {initial_data_version_name}")

✅ Data version created: initial_production_training_data_v1
   Shape: (150, 5)
   Hash: 7b49656774eb8be2...
   Source: initial_training
Initial data version created: initial_production_training_data_v1


In [31]:

print("Enhanced MLflow Pipeline with Model Selection & Hyperparameter Tuning")
print("="*70)

# Run complete workflow example
# pipeline, result = example_complete_workflow()

# Run CatBoost example
# catboost_result = example_with_catboost()

# Run retraining workflow
# initial_result, retrain_result = example_retraining_workflow()

print("\n✅ All examples completed!")
print("\nTo use with your own data:")

# 1. Initialize pipeline
pipeline = MLflowVersioningPipeline(
    experiment_name=experiment_name,
    model_name=model_name
)

# 2. Define models and hyperparameter spaces
models_dict = {
    'randomforest': RandomForestClassifier,
    'xgboost': xgb.XGBClassifier,
    'lightgbm': lgb.LGBMClassifier,
    'catboost': CatBoostClassifier
}

hyperparams_dict = {
    'randomforest': {
        'n_estimators': {'type': 'int', 'low': 50, 'high': 200},
        'max_depth': {'type': 'int', 'low': 3, 'high': 15}
    },
    'xgboost': {
        'n_estimators': {'type': 'int', 'low': 50, 'high': 200},
        'learning_rate': {'type': 'float', 'low': 0.01, 'high': 0.3}
    }
    # Add more models and parameters as needed
}

# 3. Train with model selection
result = pipeline.run_training_pipeline(
    X_train, y_train, X_test, y_test,
    models_dict=models_dict,
    hyperparams_dict=hyperparams_dict,
    metric='f1',  # or 'accuracy', 'precision', 'recall', 'roc_auc'
    n_trials=100,
    data_version=initial_data_version_name,
    description="Training for initial production model selection"
)




[I 2025-06-09 11:57:44,350] A new study created in memory with name: no-name-4905d779-d0b2-4e84-822c-357195fba787


Enhanced MLflow Pipeline with Model Selection & Hyperparameter Tuning

✅ All examples completed!

To use with your own data:
🚀 Starting Training Pipeline with Model Selection

🔍 Testing model: randomforest
   Optimizing hyperparameters with 100 trials...


[I 2025-06-09 11:57:44,881] Trial 0 finished with value: 0.9353479853479854 and parameters: {'n_estimators': 155, 'max_depth': 3}. Best is trial 0 with value: 0.9353479853479854.
[I 2025-06-09 11:57:45,483] Trial 1 finished with value: 0.9353479853479854 and parameters: {'n_estimators': 182, 'max_depth': 11}. Best is trial 0 with value: 0.9353479853479854.
[I 2025-06-09 11:57:46,004] Trial 2 finished with value: 0.9353479853479854 and parameters: {'n_estimators': 160, 'max_depth': 11}. Best is trial 0 with value: 0.9353479853479854.
[I 2025-06-09 11:57:46,263] Trial 3 finished with value: 0.9443162393162392 and parameters: {'n_estimators': 77, 'max_depth': 11}. Best is trial 3 with value: 0.9443162393162392.
[I 2025-06-09 11:57:46,545] Trial 4 finished with value: 0.9353479853479854 and parameters: {'n_estimators': 81, 'max_depth': 14}. Best is trial 3 with value: 0.9443162393162392.
[I 2025-06-09 11:57:47,051] Trial 5 finished with value: 0.9349446958270488 and parameters: {'n_estimat

   Best CV score: 0.9532
   Train f1: 1.0000
   Test f1: 1.0000

🔍 Testing model: xgboost
   Optimizing hyperparameters with 100 trials...


[I 2025-06-09 11:58:40,489] Trial 0 finished with value: 0.9442368742368743 and parameters: {'n_estimators': 200, 'learning_rate': 0.05590508409664613}. Best is trial 0 with value: 0.9442368742368743.
[I 2025-06-09 11:58:40,650] Trial 1 finished with value: 0.9442368742368743 and parameters: {'n_estimators': 102, 'learning_rate': 0.2186113040068244}. Best is trial 0 with value: 0.9442368742368743.
[I 2025-06-09 11:58:40,816] Trial 2 finished with value: 0.9442368742368743 and parameters: {'n_estimators': 101, 'learning_rate': 0.21365507393751168}. Best is trial 0 with value: 0.9442368742368743.
[I 2025-06-09 11:58:41,006] Trial 3 finished with value: 0.9442368742368743 and parameters: {'n_estimators': 103, 'learning_rate': 0.10559983973143014}. Best is trial 0 with value: 0.9442368742368743.
[I 2025-06-09 11:58:41,260] Trial 4 finished with value: 0.9442368742368743 and parameters: {'n_estimators': 162, 'learning_rate': 0.1446405230236645}. Best is trial 0 with value: 0.944236874236874

   Best CV score: 0.9442
   Train f1: 1.0000
   Test f1: 0.9484

🔍 Testing model: lightgbm
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000033 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 112, number of used features: 4
[LightGBM] [Info] Start training from score -1.054937
[LightGBM] [Info] Start training from score -1.134980
[LightGBM] [Info] Start training from score -1.107581
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

2025/06/09 11:59:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model version 3 registered
✅ Training completed!
   Best Model: randomforest
   Model Version: 3
   Best f1: 1.0000


Registered model 'test-model' already exists. Creating a new version of this model...
Created version '3' of model 'test-model'.


In [32]:
result

{'run_id': '8d14f5c748d540e68e574ddaad8efc85',
 'model_version': 3,
 'best_model_name': 'randomforest',
 'best_params': {'n_estimators': 88, 'max_depth': 9},
 'best_score': 1.0,
 'metric': 'f1',
 'final_model': RandomForestClassifier(max_depth=9, n_estimators=88),
 'scaler': StandardScaler(),
 'model_results': {'randomforest': {'model': RandomForestClassifier(max_depth=9, n_estimators=88),
   'params': {'n_estimators': 88, 'max_depth': 9},
   'train_score': 1.0,
   'test_score': 1.0,
   'cv_score': 0.9532051282051283},
  'xgboost': {'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=None, device=None, early_stopping_rounds=None,
                 enable_categorical=False, eval_metric=None, feature_types=None,
                 feature_weights=None, gamma=None, grow_policy=None,
                 importance_type=None, interaction_constraints=None,
                 learning_r

In [34]:
# After training
run_id = result['run_id']
model_version_trained = result['model_version']

In [44]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
run = client.get_run(run_id)
print(f"\nDetails for Run ID: {run_id}")
print(f"  Data Version Used: {run.data.params.get('data_version')}")
print(f"  Best Model: {run.data.params.get('best_model')}")
print(f"  Best Metric ({run.data.params.get('metric')}): {run.data.metrics.get('best_f1')}") # Or 'best_f1', etc.
print(f"  Best Params: {run.data.params.get('best_params')}")


Details for Run ID: 8d14f5c748d540e68e574ddaad8efc85
  Data Version Used: initial_production_training_data_v1
  Best Model: randomforest
  Best Metric (f1): 1.0
  Best Params: {'n_estimators': 88, 'max_depth': 9}


In [38]:
run

<Run: data=<RunData: metrics={'best_f1': 1.0,
 'catboost_test_f1': 1.0,
 'catboost_train_f1': 1.0,
 'lightgbm_test_f1': 0.9751724137931035,
 'lightgbm_train_f1': 1.0,
 'randomforest_test_f1': 1.0,
 'randomforest_train_f1': 1.0,
 'xgboost_test_f1': 0.9483643892339545,
 'xgboost_train_f1': 1.0}, params={'best_model': 'randomforest',
 'best_params': "{'n_estimators': 88, 'max_depth': 9}",
 'catboost_best_params': "{'verbose': False}",
 'data_version': 'initial_production_training_data_v1',
 'lightgbm_best_params': '{}',
 'metric': 'f1',
 'model_version': '3',
 'n_trials': '100',
 'randomforest_best_params': "{'n_estimators': 88, 'max_depth': 9}",
 'test_shape': '(38, 4)',
 'train_shape': '(112, 4)',
 'xgboost_best_params': "{'n_estimators': 200, 'learning_rate': "
                        '0.05590508409664613}'}, tags={'best_model': 'randomforest',
 'mlflow.log-model.history': '[{"run_id": "8d14f5c748d540e68e574ddaad8efc85", '
                             '"artifact_path": "model", "utc_ti

In [45]:
model_version_number = result['model_version']

In [ ]:
model_uri = f"models:/your_production_classifier/{model_version_number}"
print(f"Model URI to serve: {model_uri}")

In [ ]:

# ===== USAGE EXAMPLES =====

def example_complete_workflow():
    """Complete workflow example with model selection and hyperparameter tuning"""

    print("🎯 COMPLETE WORKFLOW WITH MODEL SELECTION")
    print("="*60)

    # Initialize pipeline
    pipeline = MLflowVersioningPipeline()

    # 1. Create training data
    print("\n1️⃣ Creating Training Data...")
    X_train = pd.DataFrame({
        'feature1': np.random.normal(0, 1, 800),
        'feature2': np.random.normal(2, 1.5, 800),
        'feature3': np.random.uniform(-1, 1, 800),
        'feature4': np.random.exponential(1, 800)
    })
    y_train = np.random.choice([0, 1], 800, p=[0.6, 0.4])

    X_test = pd.DataFrame({
        'feature1': np.random.normal(0, 1, 200),
        'feature2': np.random.normal(2, 1.5, 200),
        'feature3': np.random.uniform(-1, 1, 200),
        'feature4': np.random.exponential(1, 200)
    })
    y_test = np.random.choice([0, 1], 200, p=[0.6, 0.4])

    # 2. Define models and hyperparameter spaces
    models_dict = {
        'randomforest': RandomForestClassifier,
        'xgboost': xgb.XGBClassifier,
        'lightgbm': lgb.LGBMClassifier
    }

    hyperparams_dict = {
        'randomforest': {
            'n_estimators': {'type': 'int', 'low': 50, 'high': 200},
            'max_depth': {'type': 'int', 'low': 3, 'high': 15},
            'min_samples_split': {'type': 'int', 'low': 2, 'high': 10}
        },
        'xgboost': {
            'n_estimators': {'type': 'int', 'low': 50, 'high': 200},
            'max_depth': {'type': 'int', 'low': 3, 'high': 10},
            'learning_rate': {'type': 'float', 'low': 0.01, 'high': 0.3},
            'subsample': {'type': 'float', 'low': 0.6, 'high': 1.0}
        },
        'lightgbm': {
            'n_estimators': {'type': 'int', 'low': 50, 'high': 200},
            'max_depth': {'type': 'int', 'low': 3, 'high': 10},
            'learning_rate': {'type': 'float', 'low': 0.01, 'high': 0.3},
            'num_leaves': {'type': 'int', 'low': 10, 'high': 100}
        }
    }

    # 3. Train with model selection and hyperparameter tuning
    print("\n2️⃣ Training with Model Selection...")
    training_result = pipeline.run_training_pipeline(
        X_train, y_train, X_test, y_test,
        models_dict=models_dict,
        hyperparams_dict=hyperparams_dict,
        metric='f1',
        n_trials=50
    )

    # 4. Make predictions
    print("\n3️⃣ Making Predictions...")
    new_data = pd.DataFrame({
        'feature1': np.random.normal(0, 1, 100),
        'feature2': np.random.normal(2, 1.5, 100),
        'feature3': np.random.uniform(-1, 1, 100),
        'feature4': np.random.exponential(1, 100)
    })

    predictions, pred_version = pipeline.make_predictions_with_logging(new_data)

    # 5. Show pipeline status
    print("\n4️⃣ Pipeline Status...")
    status = pipeline.get_pipeline_status()
    print(f"Data Versions: {status['data_versions']['total']}")
    print(f"Model Versions: {status['model_versions']['total']}")
    print(f"Latest Model: v{status['model_versions']['latest']}")
    print(f"Best Model: {training_result['best_model_name']}")
    print(f"Best Score: {training_result['best_score']:.4f}")

    return pipeline, training_result



In [ ]:
def example_with_catboost():
    """Example with CatBoost included"""

    print("\n🐱 EXAMPLE WITH CATBOOST")
    print("="*40)

    pipeline = MLflowVersioningPipeline()

    # Create data
    X_train = pd.DataFrame({
        'feature1': np.random.normal(0, 1, 600),
        'feature2': np.random.normal(2, 1.5, 600),
        'feature3': np.random.uniform(-1, 1, 600)
    })
    y_train = np.random.choice([0, 1], 600, p=[0.7, 0.3])

    X_test = pd.DataFrame({
        'feature1': np.random.normal(0, 1, 150),
        'feature2': np.random.normal(2, 1.5, 150),
        'feature3': np.random.uniform(-1, 1, 150)
    })
    y_test = np.random.choice([0, 1], 150, p=[0.7, 0.3])

    # Models with CatBoost
    models_dict = {
        'randomforest': RandomForestClassifier,
        'xgboost': xgb.XGBClassifier,
        'catboost': CatBoostClassifier
    }

    hyperparams_dict = {
        'randomforest': {
            'n_estimators': {'type': 'int', 'low': 50, 'high': 150},
            'max_depth': {'type': 'int', 'low': 5, 'high': 15}
        },
        'xgboost': {
            'n_estimators': {'type': 'int', 'low': 50, 'high': 150},
            'max_depth': {'type': 'int', 'low': 3, 'high': 8},
            'learning_rate': {'type': 'float', 'low': 0.05, 'high': 0.25}
        },
        'catboost': {
            'iterations': {'type': 'int', 'low': 50, 'high': 150},
            'depth': {'type': 'int', 'low': 4, 'high': 8},
            'learning_rate': {'type': 'float', 'low': 0.05, 'high': 0.25}
        }
    }

    # Train with ROC-AUC metric
    result = pipeline.run_training_pipeline(
        X_train, y_train, X_test, y_test,
        models_dict=models_dict,
        hyperparams_dict=hyperparams_dict,
        metric='roc_auc',
        n_trials=30
    )

    print(f"Best Model: {result['best_model_name']}")
    print(f"Best ROC-AUC: {result['best_score']:.4f}")

    return result




[I 2025-06-07 03:13:36,335] A new study created in memory with name: no-name-ae6005bd-36e2-4d01-8ae0-71a68603e8c7
[W 2025-06-07 03:13:36,338] Trial 0 failed with parameters: {'n_estimators': 155, 'max_depth': 14} because of the following error: TypeError("MLflowVersioningPipeline._get_metric_function() missing 1 required positional argument: 'y_train'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-26-7e37e2acb723>", line 87, in objective
    cv=5, scoring=self._get_metric_function(metric)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: MLflowVersioningPipeline._get_metric_function() missing 1 required positional argument: 'y_train'
[W 2025-06-07 03:13:36,340] Trial 0 failed with value None.


Enhanced MLflow Pipeline with Model Selection & Hyperparameter Tuning

✅ All examples completed!

To use with your own data:
🚀 Starting Training Pipeline with Model Selection

🔍 Testing model: randomforest
   Optimizing hyperparameters with 100 trials...


TypeError: MLflowVersioningPipeline._get_metric_function() missing 1 required positional argument: 'y_train'

In [ ]:
# 4. Make predictions
predictions, pred_version = pipeline.make_predictions_with_logging(X_new)


In [ ]:
# 5. Retrain with new data
retrain_result = pipeline.retrain_with_new_data(
    new_data,
    models_dict=models_dict,
    hyperparams_dict=hyperparams_dict,
    metric='f1'
)
